# Librerie
caricamento librerie

In [1]:
import pandas as pd

### Carica DataSet
games_march2025_cleaned

In [62]:
dataset = pd.read_csv("DataSet/games_march2025_cleaned.csv")

#dataset

# Data Understanding

In [ ]:
dataset.info()

score_rank troppi null 
metacritic_url non significativo



In [ ]:
dataset.describe()
#statistiche del dataset

* Eliminiamo colonna 'appid', id interno al dataset di steam, non utile alla prediizione
* Eliminiamo colonna della descrizione e raccomandazioni, sono testuali e non utili alla predizione.
    * 'detailed_description' , 'about_the_game', 'short_description', 'reviews'
* Eliminiamo url dell'immagine del gioco
    * 'header_image'
* Eliminiamo colonne testuali per supporto clienti e url vari
    * 'website', 'support_url', 'support_email', 'metacritic_url', 'notes'
* Eliminiamo colonne per trailer video e foto del gioco
    * 'movies' , 'screenshots'
* Elimino i punteggi che hanno troppi valori null
    * 'user_score', 'score_rank'
* Elimino le liste delle lingue supportate e dei pacchetti che non danno utilità
    * 'supported_languages', 'full_audio_languages', 'packages'
* Elimino numero di dlc e sconti che non danno informazioni o con molti valori nulli
    * 'discount', 'dlc_count', 'tags'

In [63]:
# drop elimina colonna, per eliminare più colonne contemporaneamente [ , ]
dataset.drop('appid', axis=1, inplace=True)
dataset.drop('name', axis=1, inplace=True)
dataset.drop(['detailed_description','about_the_game', 'short_description', 'reviews', 'header_image'], axis=1, inplace=True)
dataset.drop(['website', 'support_url', 'support_email', 'metacritic_url', 'notes'], axis=1, inplace=True)
dataset.drop(['screenshots','movies'], axis=1, inplace=True)
dataset.drop(['user_score', 'score_rank', 'metacritic_score'], axis=1, inplace=True)
dataset.drop(['supported_languages', 'full_audio_languages', 'packages', 'tags'], axis=1, inplace=True)
dataset.drop(['discount', 'dlc_count'], axis=1, inplace=True)

In [ ]:
#dataset.describe(include=['O'])
dataset['user_score'].value_counts()
dataset['metacritic_score'].value_counts()
dataset['score_rank'].value_counts()
dataset['positive'].value_counts()
dataset.info()

### Salvo su nuovo dataset da poter usare comodamente

In [64]:
# salvo dataset modificato in uno nuovo per poter ripartire da lì
dataset.to_csv('DataSet/games1.csv', index=False)


##### Salvo le modifiche in un nuovo dataset per poterlo utilizzare
* Inizializzare da QUI

In [65]:
#NUOVO DATASET CON MODIFICHE

ds = pd.read_csv("DataSet/games1.csv")

* Creo nuova variabile che include differenza tra positivi e negativi
    * elimino le due colonne di positivi e negativi

In [66]:
ds['absolute_score'] = ds['positive'] - ds['negative']

ds.drop(['positive', 'negative'], axis=1, inplace=True)
ds.to_csv('DataSet/games1.csv', index=False)


### Taglio dataset sugli ultimi 10 anni:
* taglio tutti i giochi usciti prima del 2015
    * dataset arriva fino al 2025

In [67]:
ds['release_date'] = pd.to_datetime(ds['release_date'], format='%Y-%m-%d', errors='coerce')

cutoff_date = pd.Timestamp('2015-01-01')

# Filtriamo solo le righe dal 2015 in poi
ds = ds[ds['release_date'] >= cutoff_date]

print(f"Righe rimanenti: {len(ds)}")


Righe rimanenti: 86847


### Variabile Target num giocatori che hanno acquistato il gioco
* per decretare successo del gioco

In [68]:
ds['estimated_owners'] = ds['estimated_owners'].apply(lambda x: x.split('-')[0])
ds['estimated_owners'].value_counts()

estimated_owners
0            67288
20000         9144
50000         4130
100000        2586
200000        2022
500000         833
1000000        441
2000000        263
5000000         78
10000000        34
20000000        18
50000000        10
Name: count, dtype: int64

### Raggruppamento
* Faccio raggruppamento
    * raggruppo per 4 valori quindi 4 intervalli

In [69]:
# dopo il taglio era diventato stringhe
ds['estimated_owners'] = pd.to_numeric(ds['estimated_owners'], errors='coerce')

intervalli = [0, 5000, 50000, 500000, float('inf')]
etichette = ['insuccesso', 'basso_successo', 'medio_successo', 'alto_successo']

ds['success_class'] = pd.cut(ds['estimated_owners'], bins=intervalli, labels=etichette, include_lowest=True)

print(ds['success_class'].value_counts())

success_class
insuccesso        67288
basso_successo    13274
medio_successo     5441
alto_successo       844
Name: count, dtype: int64


### Eseguo Undersampling
* Dataset troppo sbilanciato meglio eseguire undersampling e portare tutte le classi di successo allo stesso valore della più piccola.

In [70]:
from imblearn.under_sampling import RandomUnderSampler

# X = tutte le colonne tranne la target
X = ds.drop(columns=['success_class'])

# y = la target
y = ds['success_class']

undersampler = RandomUnderSampler( sampling_strategy='auto', random_state=42)

# Eseguo Undersampling
X_resampled, y_resampled = undersampler.fit_resample(X, y)

# Ricombino in un nuovo DataFrame
ds = pd.concat([X_resampled, y_resampled], axis=1)

# elimino colonna 'estimated_owners' non serve più - sostituita da 'success_class'
ds.drop('estimated_owners', axis=1, inplace=True)

print("Distribuzione originale:")
print(y.value_counts())

print("\nDistribuzione dopo undersampling:")
print(y_resampled.value_counts())


Distribuzione originale:
success_class
insuccesso        67288
basso_successo    13274
medio_successo     5441
alto_successo       844
Name: count, dtype: int64

Distribuzione dopo undersampling:
success_class
insuccesso        844
basso_successo    844
medio_successo    844
alto_successo     844
Name: count, dtype: int64


### Elimino colonne che hanno troppe variabili nulle e che vanno a squilibrare il dataset
* 'pct_pos_total', 'pct_pos_recent', 'num_reviews_total', 'num_reviews_recent', 'peak_ccu'
    * hanno varibile nulla => 800

In [71]:
print(ds['num_reviews_recent'].value_counts())
ds.drop(['pct_pos_total', 'pct_pos_recent', 'num_reviews_total', 'num_reviews_recent', 'peak_ccu'] , axis=1, inplace=True)

ds.to_csv('DataSet/games1.csv', index=False)

num_reviews_recent
-1       2154
 10        25
 11        23
 12        23
 13        23
         ... 
 854        1
 108        1
 889        1
 139        1
 1101       1
Name: count, Length: 577, dtype: int64


# Lettura da games1.csv

In [9]:
ds = pd.read_csv("DataSet/games1.csv")
#print(ds)

# Encoding delle variabili
* Ci sono colonne che contengono liste di variabili.
    * Eseguo Encoding su 'genres' e 'categories'

In [10]:
import ast
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

# Per 'genres'
# Converto le stringhe in liste
ds['genres'] = ds['genres'].apply(ast.literal_eval)

# mlb classe contiene ordine dei generi
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(ds['genres'])

# Salvo in una colonna come array
ds['genres_vector'] = list(genres_encoded)

# Per 'categories'
ds['categories'] = ds['categories'].apply(ast.literal_eval)

mlb_categories = MultiLabelBinarizer()
categories_encoded = mlb_categories.fit_transform(ds['categories'])

ds['categories_vector'] = list(categories_encoded)

# esempio
print(ds['genres_vector'].iloc[0])
print(ds['categories_vector'].iloc[0])


[0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [12]:
ds.drop(['genres', 'categories'], axis=1, inplace=True)

In [13]:
ds.to_csv('DataSet/games1.csv', index=False)